In [2]:
import logging
import traceback
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.6.0


In [3]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22' #'d128f140-94e6-4175-87a7-954b9d27db16'  #
workspace_name = 'lisal-amlservice' #'lisal-IgniteTest' #
resource_group = 'lisal-dev'


In [4]:
from azureml.core.workspace import Workspace
from azureml.core.workspace import UserErrorException
from azureml.core.authentication import InteractiveLoginAuthentication

# test if code above (option 1) failed
try:
    auth = InteractiveLoginAuthentication()

    ws = Workspace.get(
        subscription_id = subscription_id,
        name            = workspace_name,  
        resource_group  = resource_group,
        auth=auth
    )

    print("Successfully connected to workspace.")

except UserErrorException:
    logging.critical("Could not connect using the provided identifiers\n{}".format(traceback.format_exc()))

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
Successfully connected to workspace.


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.contrib.compute import AmlWindowsCompute

compute_name = "windows-cpu"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlWindowsCompute.provisioning_configuration(
        vm_size = "STANDARD_D2_V2",
        vm_priority='dedicated',
        min_nodes = 1,
        max_nodes = 1,
        admin_username='lisal',
        admin_user_password='Greenview_1016',
        remote_login_port_public_access='Enabled'
    )
    
    # create the windows computecluster
    compute_target = AmlWindowsCompute.create(ws, compute_name, provisioning_config)
    
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=15)
    
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlWindowsCompute wait for completion finished
There were errors reported from AmlWindowsCompute:
[{'error': {'code': 'ClusterCoreQuotaReached', 'message': 'Operation results in exceeding quota limits of Total Cluster Dedicated Regional vCPUs. Maximum allowed: 100, Current in use: 100, Additional requested: 2. Please contact support to increase the quota for resource type Total Cluster Dedicated Regional vCPUs'}}]
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-06-02T02:31:20.648000+00:00', 'errors': [{'error': {'code': 'ClusterCoreQuotaReached', 'message': 'Operation results in exceeding quota limits of Total Cluster Dedicated Regional vCPUs. Maximum allowed: 100, Current in use: 100, Additional requested: 2. P